In [1]:
# Дан файл с некоторыми данными. Формат файла – произвольный. 
# Данные некоторым образом связаны с теми, что были добавлены в первом задании. 
# Необходимо проанализировать и установить связь между таблицами. 
# Создать таблицу и наполнить ее прочитанными данными из файла. 
# Реализовать и выполнить 3 запроса, где используется связь между таблицами.

In [27]:
import pandas as pd
import sqlite3
import pickle
import json

In [28]:
with open('task_2_var_63_subitem.pkl', 'rb') as file:
    plk_data = pickle.load(file)
plk_data[0:3]    

[{'name': 'Мемориал Таля 1979', 'place': 7, 'prise': 15625},
 {'name': 'Гашиш 2004', 'place': 13, 'prise': 5005},
 {'name': 'Вена 1966', 'place': 2, 'prise': 11750000}]

In [29]:
def connect_to_db(file):
    connection = sqlite3.connect(file)
    connection.row_factory = sqlite3.Row
    return connection

In [30]:
# Создание таблицы в базе данных
def create_table():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS properties_2
                  (id INTEGER PRIMARY KEY AUTOINCREMENT,
                   name TEXT, 
                   place INT,
                prise INT)''')
    conn.commit()
    conn.close()

In [31]:
# Чтение данных из файла в таблицу
def insert_data():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    # Чтение данных из файла task_1_var_63_item.pkl
    with open('task_2_var_63_subitem.pkl', 'rb') as file:
        json_data = pickle.load(file)
        
    rows = []
    for item in json_data:
        row = {
            'name': item['name'], 
            'place': item['place'], 
            'prise': item['prise']
        }
        rows.append(row)    
    # Запись данных в таблицу
    c.executemany("INSERT OR IGNORE INTO properties_2 (name, place, prise) VALUES (:name, :place, :prise)", rows)
    # Сохранение изменений
    conn.commit()
    conn.close() 

In [41]:
# Объединить две таблицы properties и properties_2 по полю name


# Получить все свойства и их рейтинги
def oll_properties(): 
    # Устанавливаем соединение базой данных
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    # Выполняем запрос
    cursor.execute('''
        SELECT p.name, p.system, p_2.prise
        FROM properties p
        INNER JOIN properties_2 p_2 ON p.name = p_2.name
    ''')
    # Получаем результат запроса
    results = cursor.fetchall()

    # Создаем список для хранения результатов
    property_list = []

    # Проходимся по каждой строке и добавляем данные в список
    for row in results:
        property_data = {
            'Система': row[0],
            'Цена': row[1]
        }
        property_list.append(property_data)

    # Закрываем соединение
    conn.close()

    # Записываем список в файл JSON
    with open('oll_properties_63_2.json', 'w', encoding='utf-8') as json_file:
        json.dump(property_list, json_file, ensure_ascii=False)

In [47]:
# Получить все свойства с местом и ценой выше 117500
def prise_properties(): 
    # Устанавливаем соединение базой данных
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    # Выполняем запрос
    cursor.execute('''
        SELECT p.name, p_2.place, p_2.prise
        FROM properties p
        JOIN properties_2 p_2 ON p.name = p_2.name
        WHERE p_2.prise > 117500
    ''')
    # Получаем результат запроса
    results = cursor.fetchall()

    # Создаем список для хранения результатов
    property_list = []

    # Проходимся по каждой строке и добавляем данные в список
    for row in results:
        property_data = {
            'Место': row[0],
            'Цена': row[1]
        }
        property_list.append(property_data)

    # Закрываем соединение
    conn.close()

    # Записываем список в файл JSON
    with open('prise_properties_63_2.json', 'w', encoding='utf-8') as json_file:
        json.dump(property_list, json_file, ensure_ascii=False)

In [48]:
# Получить все свойства с ценой и местом выше 5
def five_properties(): 
    # Устанавливаем соединение базой данных
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    # Выполняем запрос
    cursor.execute('''
        SELECT p.name, p_2.place
        FROM properties p
        JOIN properties_2 p_2 ON p.name = p_2.name
        WHERE p_2.place > 5
    ''')
    # Получаем результат запроса
    results = cursor.fetchall()

    # Создаем список для хранения результатов
    property_list = []

    # Проходимся по каждой строке и добавляем данные в список
    for row in results:
        property_data = {
            'Место': row[0],
            'Цена': row[1],
        }
        property_list.append(property_data)

    # Закрываем соединение
    conn.close()

    # Записываем список в файл JSON
    with open('five_properties_63_2.json', 'w', encoding='utf-8') as json_file:
        json.dump(property_list, json_file, ensure_ascii=False)

In [49]:
connect_to_db('database.db')
create_table()
insert_data()
oll_properties()
prise_properties()
five_properties()